In [1]:
import os
import re

import pandas as pd

In [2]:
rename_columns = {
    "Data_ID": "dataset_id",
    "IndicatorID": "indicator_id",
    "Category": "category",
    "Indicator Name": "name",
    "Data_Name": "name",
    "Indicator_Description": "description",
    "Indicator Source(s)": "source",
    "Dataset_Description": "description",
    "Dataset_Examples_ Citation": "citation",
    "Dataset_Sources_Examples_ Licenses": "licenses",
    "Dataset_Source": "source",
    "Dataset_Examples_ Notes": "notes",
    "Dataset_ Citation": "citation",
    "Dataset_Licenses": "license",
}

# ddf = pd.read_csv("datasets.csv").dropna(how="all").rename(columns=rename_columns).set_index("dataset_id")
# idf = pd.read_csv("indicators.csv").dropna(how="all").rename(columns=rename_columns).set_index("indicator_id")
file = "VALORADA_Data catalogue_CLean_Version_15.08.2025_2.xlsx"

dir = "data"

# Function to remove leading and trailing format/control characters


def clean_format_chars(val):
    if isinstance(val, str):
        # Remove leading and trailing whitespace and control characters
        return re.sub(r"^[\s\0-\x1F\x7F]+|[\s\0-\x1F\x7F]+$", "", val)
    return val


idf = (
    pd.read_excel(file, sheet_name="Page_1_CIC_Indicators_Data")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("indicator_id")
    .map(clean_format_chars)
)
ddf = (
    pd.read_excel(file, sheet_name="Page_2_Data_To_Indicators")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("dataset_id")
    .map(clean_format_chars)
)

idf["category"] = idf["category"].str.title()

## Create links to datasets

In [3]:
ddf_link_cols = [
    c for c in ddf.columns if c.lower().startswith("ind") and "id" in c.lower()
]
# ddf[[c for c in ddf.columns if c not in link_cols]].to_csv("datasets.csv", index=True)
ddf_links = ddf[
    ddf_link_cols
]  # .reset_index().melt(["dataset_id"], value_name="dataset_id").drop(columns=["variable"])

idf_link_cols = [
    c for c in idf.columns if c.lower().startswith("data_") and "id" in c.lower()
]
# idf[[c for c in idf.columns if c not in link_cols]].to_csv("indicators.csv", index=True)
idf_links = idf[
    idf_link_cols
]  # .reset_index().melt(["indicator_id"], value_name="dataset_id").drop(columns=["variable"])

In [4]:
cols = ["indicator_id", "dataset_id"]

idf_links.reset_index().melt(
    id_vars=["indicator_id"],
    value_vars=idf_link_cols,
    value_name="dataset_id",
).replace("xxx", pd.NA).dropna(subset=["dataset_id"]).drop(columns=["variable"])[
    cols
].sort_values(
    by=["indicator_id", "dataset_id"]
)[
    cols
].to_csv(
    os.path.join(dir, "links_indicator_to_data.csv"), index=False
)

In [5]:
ddf_links.reset_index().melt(
    id_vars=["dataset_id"],
    value_vars=ddf_link_cols,
    value_name="indicator_id",
).replace("xxx", pd.NA).dropna(subset=["indicator_id"]).drop(columns=["variable"])[
    cols
].sort_values(
    by=["indicator_id", "dataset_id"]
).to_csv(
    os.path.join(dir, "links_data_to_indicator.csv"), index=False
)

In [6]:
ddf[[c for c in ddf.columns if c in rename_columns.values()]].to_csv(
    os.path.join(dir, "datasets.csv"), index=True
)
idf[[c for c in idf.columns if c in rename_columns.values()]].to_csv(
    os.path.join(dir, "indicators.csv"), index=True
)

In [7]:
# group links for airtable
links = pd.read_csv(os.path.join(dir, "links_indicator_to_data.csv"))
links = (
    links.groupby("indicator_id")["dataset_id"]
    .apply(lambda x: ",".join(x))
    .reset_index()
)
links.to_csv(os.path.join(dir, "links_grouped_indicator_to_data.csv"), index=False)

# group links for airtable
links = pd.read_csv(os.path.join(dir, "links_data_to_indicator.csv"))
links = (
    links.groupby("indicator_id")["dataset_id"]
    .apply(lambda x: ",".join(x))
    .reset_index()
)
links.to_csv(os.path.join(dir, "links_grouped_data_to_indicator.csv"), index=False)

## Create links to cic

In [8]:
cols = [
    "CIC.1. Urban: Degradation of urban ecosystems due to urban warming and drought",
    "CIC.2. Urban: High temperatures affecting people’s health and the public transport system",
    "CIC.3. Rural: Increasing temperatures and drought affecting the agricultural sector",
    "CIC.4. Rural: drought and heat increasing the risk of wildfires",
    "CIC.5. Hydrogeologic affectation on rural livelihoods, rural infrastructure and agricultural means of production",
]

indicator = pd.read_excel(file, sheet_name="Page_1_CIC_Indicators_Data").set_index(
    "IndicatorID"
)[cols]

# Convert the wide CIC indicator table to long (tidy) format
# Assumes `indicator` DataFrame as created above (index = IndicatorID, columns = CIC.* headers)

indicator_long = (
    indicator.reset_index()  # bring IndicatorID back as a column
    .melt(
        id_vars="IndicatorID", var_name="cic_full", value_name="value"
    )  # value = cell content (e.g. marker like 'x')
    .query("value == 'yes'")  # keep only marked links
    .replace(["xxx", "", " "], pd.NA)  # treat placeholders as missing
    .dropna(subset=["value"])  # keep only real links
)


# Extract a short CIC code (e.g. CIC.1) and a cleaned title
indicator_long["cic_id"] = indicator_long["cic_full"].str.extract(r"^(CIC\.\d+)")
indicator_long["cic_title"] = indicator_long["cic_full"].str.replace(
    r"^CIC\.\d+\.\s*", "", regex=True
)

# # Reorder / rename columns
indicator_long = indicator_long.rename(columns={"IndicatorID": "indicator_id"})
indicator_long = indicator_long[["indicator_id", "cic_id", "cic_title", "value"]]

# # Save
# indicator_long.to_csv(os.path.join(dir, "indicator_cic_links.csv"), index=False)
indicator_long[["indicator_id", "cic_id"]].to_csv(
    "data/links_indicator_to_cic.csv", index=False
)